In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Hi everyone,
I am not new to kaggle community, but I just took my start up for working on kaggle regularly. I am going to work on this Cat-vs-dog competition, which is not very difficult problem as our images are very well arranged with good naming convention. So we will further arrange the pictures in respective directories so that we can benefit from ***ImageDataGenerator*** class API to get the image batches and their labels from directory names cats, dogs etc. , which is made very easy by keras Class method flow_from_directory. we will see this action in a moment 

First thing first , we will unzip our picture data as show below, unzip -q means , this unzip will be done in quiet mode, so as not show the full uncompressing logs

In [ ]:
!unzip -q ../input/dogs-vs-cats-redux-kernels-edition/train.zip

In [ ]:
ls -l -t /kaggle/working/train | head -3

In [ ]:
!unzip -q ../input/dogs-vs-cats-redux-kernels-edition/test.zip

check the naming pattern of our files with the follwing command:

In [ ]:
ls -l -t /kaggle/working/test | head -3

That shows pattern *cat.****.jpg* , likewise *dog.****.jpg* in the file names. So here we can move the files in to two new folders cat and dog so that our keras API can make use of directory names to label our image batches

In [ ]:
cd /kaggle/working/train


In [ ]:
mkdir cat dog

move all dog images to dog directory

In [ ]:
mv  ./dog.* ./dog


move all cat images to cat directory

In [ ]:
mv ./cat.* ./cat

In [ ]:
cd /kaggle/working/test


In [ ]:
ls -l -t /kaggle/working/test | head -3

In test directory the image names do not contain any names like cat or dog, wo do not know the picture is of cat or dog without opeing it. That is why we will make predictions on testing data after our model is fully  trained on trainging data and validated on validation data.


Let us start making our image batches for training and validation subset using ImageGenerator class.

It has three very important benefits according to my knowledge:
1. It can infer label names from the folder / directory name
2. Image generator object can be used directly in the fit method for training and fit method will directly iterate through the batches of images during training and validation epochs
3. It can augment your train and validation image data on the runtime of the model, and exposes different orientations of an image in space, rotation, hrozontal shifting, vertical shifting etc, so that we transform each image in a way that our model will hardly able to identify  it as the same image being sent for training again and again

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
val_batch = 10
train_batch = 32
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.4,
        zoom_range=0.3,
        validation_split=0.30,
        horizontal_flip=True,
        )
train_generator = train_datagen.flow_from_directory(
        '/kaggle/working/train',
        target_size=(130, 130),
        batch_size=train_batch,
        class_mode='binary',
        subset = 'training',
        color_mode = 'rgb',
        shuffle = True,
        )
validation_generator = train_datagen.flow_from_directory(
        '/kaggle/working/train',
        target_size=(130, 130),
        batch_size=val_batch,
        class_mode='binary',
        subset = 'validation',
        color_mode = 'rgb',
        shuffle= True)

As you can see that ImageGenerator class has done all the job for us , which has to be done manually by loops otherwise.It has returned us Image generator object which will yield batches of images and the respective labels in tuples. The images has been segmented in two classes for training 17500 ( 70%) of total 25000 images, where as validation set has 7500 (30 %) of total 25000 images.We can see example of some image batches and class labels like below:

In [ ]:
print(train_generator.class_indices)

We can see train generator elements how does it look like. Lets see the first element of the train_generator.It will return a tuple of 32 images and 32 labels. Because our each training batch containes 32 images in it as specified by variable *train_batch = 32*

In [ ]:
print(train_generator[0])

our first batch contains 32 images and 32 labels

In [ ]:
print(train_generator[0][0].shape)
print(train_generator[0][1].shape)

you can observe here the inside out of image generator object. as our batch size for training is 32 and total images were 17500. Hence 17500 / 32 = 546.8 batches, lets see that our 547th batch got how many pictures in it? Interesting isnt it ?

In [ ]:
print(train_generator[546][0].shape)
print(train_generator[546][1].shape)

decimal value in 546.8 i.e. 0.8 shows there were 28 pictures left in the end, and our final batch has 28 pictures in it, whereas all other batches from 0 ~ 545 have 32 pictures and respective labels in it. likwise you can claculate batches in validation dataset.

In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Flatten
my_base_model = keras.applications.DenseNet201(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(130, 130, 3),
    include_top=False,
)  # Do not include the ImageNet classifier at the top.

# Freeze the base_model
my_base_model.trainable = False

# Create new model on top
my_inputs = keras.Input(shape=(130, 130, 3))
# x = data_augmentation(inputs)  # Apply random data augmentation

# Pre-trained Xception weights requires that input be scaled
# from (0, 255) to a range of (-1., +1.), the rescaling layer
# outputs: `(inputs * scale) + offset`
# scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
# x = scale_layer(x)

# The base model contains batchnorm layers. We want to keep them in inference mode
# when we unfreeze the base model for fine-tuning, so we make sure that the
# base_model is running in inference mode here.
x = my_base_model(my_inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
# x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
# x = keras.layers.MaxPooling2D()(x)
x = keras.layers.Dense(128, activation='relu')(x)
x = keras.layers.Dense(128, activation='relu')(x)
my_outputs = keras.layers.Dense(1, activation='sigmoid')(x)
my_model = keras.Model(my_inputs,my_outputs)

my_model.summary()

In [ ]:
my_model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=['accuracy']
)
early_stopping  = keras.callbacks.EarlyStopping(min_delta=0.001, patience=5)

epochs = 30
my_model.fit(train_generator, 
             epochs=epochs, 
             validation_data=validation_generator,
            callbacks=[early_stopping])

In [ ]:
test_datagen = ImageDataGenerator(
    rescale = 1./255
)
test_generator = test_datagen.flow_from_directory(
        '/kaggle/input/dogsvscatsmytestdata/training_set/',
        target_size = (130,130),
        batch_size = 32,
        class_mode='binary',
        color_mode = 'rgb',
)

In [ ]:
my_model.evaluate(test_generator)